## 鏈式 prompt, chaining prompts 基本式

### case: 翻譯寫作

翻譯參考自: https://twitter.com/dotey/status/1710106195640398056

類似的改寫思路:

* 作文反思改寫 https://twitter.com/mattshumer_/status/1700169043406123294
* Chain of Density 摘要法 https://twitter.com/vimota/status/1702503466994982914 和 https://www.facebook.com/minshiTsai/posts/7568225296537280 (中文說明)


In [3]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [2]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [4]:
import requests
import json
from pprint import pp

In [5]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, format_type=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if format_type:
    payload["response_format"] =  { "type": format_type }

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## 第一輪: 整體粗翻

In [6]:
# https://blog.google/technology/ai/google-gemini-next-generation-model-february-2024/#sundar-note
text = """ Introducing Gemini 1.5
By Demis Hassabis, CEO of Google DeepMind, on behalf of the Gemini team

This is an exciting time for AI. New advances in the field have the potential to make AI more helpful for billions of people over the coming years. Since introducing Gemini 1.0, we’ve been testing, refining and enhancing its capabilities.

Today, we’re announcing our next-generation model: Gemini 1.5.

Gemini 1.5 delivers dramatically enhanced performance. It represents a step change in our approach, building upon research and engineering innovations across nearly every part of our foundation model development and infrastructure. This includes making Gemini 1.5 more efficient to train and serve, with a new Mixture-of-Experts (MoE) architecture.

The first Gemini 1.5 model we’re releasing for early testing is Gemini 1.5 Pro. It’s a mid-size multimodal model, optimized for scaling across a wide-range of tasks, and performs at a similar level to 1.0 Ultra, our largest model to date. It also introduces a breakthrough experimental feature in long-context understanding.

Gemini 1.5 Pro comes with a standard 128,000 token context window. But starting today, a limited group of developers and enterprise customers can try it with a context window of up to 1 million tokens via AI Studio and Vertex AI in private preview.

As we roll out the full 1 million token context window, we’re actively working on optimizations to improve latency, reduce computational requirements and enhance the user experience. We’re excited for people to try this breakthrough capability, and we share more details on future availability below.

These continued advances in our next-generation models will open up new possibilities for people, developers and enterprises to create, discover and build using AI.
"""


messages1 = [
    {
    "role": "system",
    "content": """
            You are Translator, an AI who is skilled in translating English to Chinese Mandarin Taiwanese fluently.
            Your task is to translate an article or part of the full article which will be provided to you after you acknowledge
            this message and say you’re ready.
            Constraints:
            * Do not change any of the wording in the text in such a way that the original meaning is changed unless you are fixing typos or correcting the article.
            * Do not chat or ask.
            * Do not explain any sentences, just translate or leave them as they are.
            * When you translate a quote from somebody, please use 「」『』 instead of ""

            Pleases always respond in Chinese Mandarin Taiwanese and Taiwan terms.
            When mixing Chinese and English, add a whitespace between Chinese and English characters.
    """
    },
    {"role": "user", "content": text}
]

result1 = get_completion(messages1, model="gpt-3.5-turbo-1106", max_tokens=4096)
print(result1)

我們很高興地宣布，Google DeepMind的CEO Demis Hassabis代表Gemini團隊介紹了Gemini 1.5。

這對於人工智慧來說是令人興奮的時刻。該領域的新進展有潛力在未來幾年使人工智慧對數十億人更有幫助。自Gemini 1.0推出以來，我們一直在測試、改進和增強其功能。

今天，我們宣布推出我們的下一代模型：Gemini 1.5。

Gemini 1.5大幅提升了性能。它代表了我們方法的一個重大變革，建立在幾乎我們基礎模型開發和基礎設施的每個部分的研究和工程創新之上。這包括使Gemini 1.5更有效地進行訓練和服務，並採用了新的專家混合（MoE）架構。

我們首次推出的Gemini 1.5模型是Gemini 1.5 Pro，用於早期測試。這是一個中等大小的多模型模型，經過優化，可擴展到各種任務，並且在性能上與迄今為止我們最大的模型1.0 Ultra相當。它還引入了一個突破性的長篇理解實驗功能。

Gemini 1.5 Pro配備標準的128,000令牌上下文窗口。但從今天開始，一小部分開發人員和企業客戶可以通過AI Studio和Vertex AI進行私人預覽，嘗試使用高達100萬令牌的上下文窗口。

隨著我們推出完整的100萬令牌上下文窗口，我們正在積極努力進行優化，以改善延遲、減少計算要求並增強用戶體驗。我們很興奮讓人們嘗試這一突破性功能，並在下面分享更多未來可用性的細節。

我們下一代模型的持續進步將為人們、開發人員和企業創造、發現和構建使用人工智慧開啟新的可能性。


## 第二輪: 細部潤飾

In [9]:
messages2 = [
    {"role": "user", 
     "content": f"""
            你是一位專業中文翻譯，擅長對翻譯結果進行二次修改和文章潤飾成通俗易懂的中文，我希望你能幫我將以下英文的中文翻譯結果重新意譯和潤色。

            * 保留特定的英文術語、數字或名字，並在其前後加上空格，例如："生成式 AI 產品"，"不超過 10 秒"。
            * 基於直譯結果重新意譯，意譯時務必對照原始英文，不要添加也不要遺漏內容，並以讓翻譯結果通俗易懂，符合中文表達習慣
            * 請輸出成台灣使用的繁體中文 zh-tw

            英文原文：
            {text}

            直譯結果：
            {result1}

            重新潤飾後：
            
        """
}
]

result2 = get_completion(messages2, model="gpt-4-turbo-preview", max_tokens=4096)
print(result2)

Google DeepMind 的 CEO Demis Hassabis 代表 Gemini 團隊，興奮地向大家介紹 Gemini 1.5。

在人工智慧的領域裡，我們正處於一個充滿激動的時代。近期的突破性進展，預示著在未來幾年內，人工智慧將能為全球數十億人帶來更多幫助。自從我們推出 Gemini 1.0 以來，我們不斷地進行測試、調整和提升，以增強其功能。

今日，我們激動地宣布我們的下一代模型：Gemini 1.5。

Gemini 1.5 帶來了顯著的性能提升。這代表了我們方法上的一大飛躍，我們在幾乎所有基礎模型開發和基礎設施的部分，都進行了研究和工程創新。這包括讓 Gemini 1.5 在訓練和運行上更加高效，並引入了全新的專家混合（MoE）架構。

我們首次推出的用於早期測試的 Gemini 1.5 模型是 Gemini 1.5 Pro。它是一款中型的多模態模型，經過優化，能夠適應廣泛的任務，其性能與我們迄今為止最大的模型 1.0 Ultra 相當。此外，它還帶來了一項在長篇內容理解上的突破性實驗功能。

Gemini 1.5 Pro 標配有 128,000 令牌的上下文窗口。但從今天起，少數開發者和企業客戶就能透過 AI Studio 和 Vertex AI 進行私人預覽，體驗高達 100 萬令牌的上下文窗口。

隨著我們全面推出 100 萬令牌的上下文窗口，我們正積極優化，以降低延遲、減少計算需求並提升用戶體驗。我們迫不及待想讓大家體驗這一突破性的功能，未來的可用性詳情也將在下方分享。

我們下一代模型的持續進步，將為人們、開發者和企業開啟使用人工智慧創造、探索和建構的新可能。


## 人類都不可能一次到位，一次到位的品質可能會比較差，

生成式模型本質上就是一個文字接龍，盡可能不要跳躍性思考，而是階段性的思考，讓模型有階段性的思考推理過程，能得到更好的結果。

好的品質需要精雕細琢: 費時費力，但是品質會更好
(latency 跟 tokens 成本也比較高)